<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">CrewAI Agents: Orchestrator-Worker Patterns</center>
</h1>

In this tutorial, we’ll build a CrewAI multi-agent system using the Orchestrator–Worker pattern. This is a powerful approach for dynamically breaking down complex tasks into smaller subtasks, assigning them to specialized agents, and combining their outputs into a cohesive result.

This pattern is especially useful when the subtasks aren’t known in advance—like drafting a multi-section report, writing modular code, or conducting structured research. The orchestrator agent takes charge of planning and delegating, while the worker agents each focus on completing their assigned part.

We’ll also integrate Phoenix to trace and debug the orchestration process. With Phoenix, you can see how the orchestrator generated tasks, how each worker executed its role, and how their outputs were assembled into the final result.

By the end of this notebook, you’ll learn how to:

- Define and create a crew of specialized agents.

- Dynamically plan and delegate subtasks across workers.

- Run and evaluate the crew’s performance on a given task.

- Trace and visualize orchestration flows using Phoenix.

For this tutorial, you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)
*   Free Phoenix Cloud account and API key (https://app.phoenix.arize.com/)

# Set up Keys and Dependencies

In [ ]:
%pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

In [ ]:
import os
from getpass import getpass

if "SERPER_API_KEY" not in os.environ:
    os.environ["SERPER_API_KEY"] = getpass("🔑 Enter your Serper API key: ")

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if "PHOENIX_COLLECTOR_ENDPOINT" not in os.environ:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint")

## Configure Tracing

In [ ]:
from phoenix.otel import register

project_name = "crewai-agents-orchestrator-workers"
tracer_provider = register(project_name=project_name, auto_instrument=True)

## Define your Agents

In [ ]:
from crewai import Agent, Crew, Task

# Define worker agents
trend_researcher = Agent(
    role="AI Trend Researcher",
    goal="Analyze current advancements in AI",
    backstory="Expert in tracking and analyzing new trends in artificial intelligence.",
    verbose=True,
)

policy_analyst = Agent(
    role="AI Policy Analyst",
    goal="Examine the implications of AI regulations and governance",
    backstory="Tracks AI policy developments across governments and organizations.",
    verbose=True,
)

risk_specialist = Agent(
    role="AI Risk Specialist",
    goal="Identify potential risks in frontier AI development",
    backstory="Focuses on safety, alignment, and misuse risks related to advanced AI.",
    verbose=True,
)

synthesizer = Agent(
    role="Synthesis Writer",
    goal="Summarize all findings into a final cohesive report",
    backstory="Expert at compiling research insights into executive-level narratives.",
    verbose=True,
)

orchestrator = Agent(
    role="Orchestrator",
    goal=(
        "Your job is to delegate research and writing tasks to the correct coworker using the 'Delegate work to coworker' tool.\n"
        "For each task you assign, you MUST call the tool with the following JSON input:\n\n"
        "{\n"
        '  "task": "Short summary of the task to do (plain string)",\n'
        '  "context": "Why this task is important or part of the report (plain string)",\n'
        '  "coworker": "One of: AI Trend Researcher, AI Policy Analyst, AI Risk Specialist, Synthesis Writer"\n'
        "}\n\n"
        "IMPORTANT:\n"
        "- Do NOT format 'task' or 'context' as dictionaries.\n"
        "- Do NOT include types or nested descriptions.\n"
        "- Only use plain strings for both.\n"
        "- Call the tool multiple times, one per coworker."
    ),
    backstory="You are responsible for assigning each part of an AI report to the right specialist.",
    verbose=True,
    allow_delegation=True,
)

## Define your Tasks

In [ ]:
# Define the initial task only for the orchestrator
initial_task = Task(
    description="Create an AI trends report. It should include recent innovations, policy updates, and safety risks. Then synthesize it into a unified summary.",
    expected_output="Assign subtasks via the DelegateWorkTool and return a final report.",
    agent=orchestrator,
)

# Set up the crew (no hierarchical process needed with delegation tools)
crew = Crew(
    agents=[trend_researcher, policy_analyst, risk_specialist, synthesizer],
    tasks=[initial_task],
    manager_agent=orchestrator,
    verbose=True,
)

# Run the full workflow
result = crew.kickoff()
print(result)

# Agent Evaluation

Here, we will evaluate the agent’s trajectory. This means checking whether the sequence of steps it took was logical, efficient, and aligned with completing the user’s request. Then, we will log those results back to Phoenix.

In [ ]:
import pandas as pd

from phoenix.client import AsyncClient

px_client = AsyncClient()
df = await px_client.spans.get_spans_dataframe(project_name=project_name)

trace_df = df.groupby("context.trace_id").agg(
    {
        "attributes.input.value": "first",
        "attributes.output.value": lambda x: " ".join(x.dropna()),
    }
)


def extract_input_content(input_value):
    try:
        if pd.isna(input_value) or input_value is None:
            return None

        # JSON string
        if isinstance(input_value, str):
            import json

            try:
                parsed = json.loads(input_value)
                inputs = parsed.get("messages", [])
                if isinstance(inputs, list) and len(inputs) > 0:
                    return inputs[0].get("content")
                return input_value
            except Exception:
                return input_value

        return input_value

    except (AttributeError, TypeError, KeyError):
        return input_value


# Apply function row by row
trace_df["attributes.input.value"] = trace_df["attributes.input.value"].apply(extract_input_content)
trace_df.head()

In [ ]:
TRAJECTORY_PERFORMANCE_PROMPT = """
You are a helpful AI bot that checks whether an AI agent's internal trajectory is accurate and effective.

You will be given:
1. You will be given an input query from a user that the agent responded to
2. The agent's actual trajectory of tool calls and responses

An accurate trajectory:
- Progresses logically from step to step
- Follows the golden trajectory where reasonable
- Shows a clear path toward completing a goal
- Is reasonably efficient (doesn't take unnecessary detours)

##

User Query:
{attributes.input.value}

Trajectory:
{attributes.output.value}

##

Your response must be a single string, either `correct` or `incorrect`, and must not include any additional text.

- Respond with `correct` if the agent's trajectory adheres to the rubric and accomplishes the task effectively.
- Respond with `incorrect` if the trajectory is confusing, misaligned with the goal, inefficient, or does not accomplish the task.
"""

In [ ]:
from phoenix.evals import OpenAIModel, llm_classify
from phoenix.trace import suppress_tracing

model = OpenAIModel(
    api_key=os.environ["OPENAI_API_KEY"],
    model="gpt-4o-mini",
    temperature=0.0,
)

rails = ["correct", "incorrect"]

with suppress_tracing():
    eval_results = llm_classify(
        dataframe=trace_df,
        template=TRAJECTORY_PERFORMANCE_PROMPT,
        model=model,
        rails=rails,
        provide_explanation=True,
        verbose=False,
        concurrency=20,
    )

eval_results["score"] = eval_results["label"].apply(lambda x: 1 if x == "correct" else 0)

In [ ]:
root_spans = df[df["parent_id"].isna()][["context.trace_id", "context.span_id"]]
eval_results = eval_results[["score", "label", "explanation"]]

trajectory_eval_df = pd.merge(trace_df, eval_results, left_index=True, right_index=True, how="left")

trajectory_eval_df = pd.merge(
    trajectory_eval_df.reset_index(), root_spans, on="context.trace_id", how="left"
).set_index("context.span_id", drop=False)

### Log Evals to Phoenix

In [ ]:
await px_client.annotations.log_span_annotations_dataframe(
    dataframe=trajectory_eval_df,
    annotation_name="TRAJECTORY PERFORMANCE",
    annotator_kind="LLM",
)

# View Final Results in Phoenix

![Evals](https://storage.googleapis.com/arize-phoenix-assets/assets/images/crewai-agents-orchestrator.png)